In [1]:
#imports
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

driver = webdriver.Chrome()

In [2]:
#retrieve the links of each historical webpage
pages = []
pages_2022 = []
URLS = ["https://web.archive.org/web/20211001000000*/https://www.vegasinsider.com/auto-racing/odds/f1/",
        "https://web.archive.org/web/20220515000000*/https://www.vegasinsider.com/auto-racing/odds/f1/"]
changed_UI = False
for URL in URLS:  
    driver.get(URL)
    time.sleep(5)
    
    # March 9, 2022 is day the UI changes, so place that and following into pages_2022
    # find_elements returns a list 
    for page in driver.find_elements(By.CSS_SELECTOR, ".month"):
        # all tables are same in 2021
        if (URL == "https://web.archive.org/web/20211001000000*/https://www.vegasinsider.com/auto-racing/odds/f1/"):
            pages.extend([day.get_attribute("href") for day in page.find_elements(By.CSS_SELECTOR, ".calendar-day a")])
            continue
            
        # searching in 2022 URL now
        if (changed_UI or page.find_element(By.CLASS_NAME, 'month-title').text == 'MAR'):
            changed_UI = True
            pages_2022.extend([day.get_attribute("href") for day in page.find_elements(By.CSS_SELECTOR, ".calendar-day a")])
        else:
            pages.extend([day.get_attribute("href") for day in page.find_elements(By.CSS_SELECTOR, ".calendar-day a")])
        
#print(pages_2022)
#print("space")
#print(pages)

In [3]:
#retrieve all of the HTML contents so its predownloaded and everything runs faster
downloadedPages = []
for page in pages:
    downloadedPages.append(requests.get(page))
    
downloadedPagesChange = []
for page in pages_2022:
    downloadedPagesChange.append(requests.get(page))

In [46]:
#march 9 2022 is the day the UI changes 
#play with pandas, figure out how to create a dataframe
df = pd.DataFrame()

for page in downloadedPages:
    #retrieve list of tables on the page
    canvas = BeautifulSoup(page.text, 'html.parser')
    allTables = canvas.findAll('table')
    
    for i in range(len(allTables)):
        if i % 2 == 0 and not allTables[i].has_attr("class"):
            try:
                title = allTables[i].find("td", {"class": "viHeaderNorm"}).text
                df[title] = None

                rows = allTables[i].findAll({"tr", "padCenter"})
                for j in range(len(rows)):
                    if j > 2:
                        dataPoints = rows[j].findAll("td")
                        currentAthlete = dataPoints[0].text
                        odds = dataPoints[1].text
                        try:
                            df.loc[currentAthlete]
                        except:
                            df.loc[currentAthlete] = None
                        df.loc[currentAthlete, title] = odds
            except:
                title = canvas.findAll("h2")[0].text
                print(title)
                df[title] = None
                rows = canvas.findAll("figure", {"class": "wp-block-table is-style-stripes"})[0]
                rows = rows.findAll("tr")
                for j in range(len(rows)):
                    if j > 2:
                        dataPoints = rows[j].findAll("td")
                        currentAthlete = dataPoints[0].text
                        odds = dataPoints[1].text
                        try:
                            df.loc[currentAthlete]
                        except:
                            df.loc[currentAthlete] = None
                        df.loc[currentAthlete, title] = odds
              #  print(rows)
            break;

df

/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/2343417806.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[currentAthlete] = None
/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/2343417806.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[currentAthlete] = None
/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/2343417806.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[currentAthlete] = None
/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/2343417806.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' i

,Odds To Win 2021 Emilia Romagna GP (4/18/21),Odds To Win 2021 F1 Portuguese GP (3/28/21),Odds To Win 2021 F1 Spanish GP (5/9/21),Odds To Win 2021 F1 Monaco GP (3/28/21),Odds To Win 2021 F1 World Championship (3/28/21),Odds To Win 2021 F1 Azerbaijan GP (6/6/21),Odds To Win 2021 F1 French GP (6/20/21),Odds To Win 2021 F1 Austrian GP (6/26/21),Odds To Win 2021 F1 Austrian GP (7/4/21),Odds To Win 2021 F1 British GP (7/18/21),Odds To Win 2021 F1 Hungarian GP (8/1/21),Odds To Win 2021 F1 Belgium GP (8/29/21),Odds To Win 2021 F1 Dutch GP (9/5/21),Odds to Win 2021 F1 Italian GP (9/12/2021),Odds to Win 2021 F1 Russian GP (9/12/2021),Odds to Win 2021 F1 Turkish GP (10/10/2021),Odds to Win 2021 F1 U.S. GP (10/24/2021),Odds to Win 2021 F1 Mexico City GP (11/5/2021),Odds to Win 2021 F1 Abu Dhabi GP (12/12/2021),Odds to Win 2022 F1 World Championship (12/12/2022)
Max Verstappen,+130,+140,+200,+140,+122,-110,+110,-148,-155,-160,-110,+110,-138,+150,+450,+100,+150,-150,+200,+225
Lewis Hamilton,+150,+160,-195,+100,-160,+250,+140,+200,+200,+185,+120,+110,+187,-120,-163,+120,-120,+187,-225,+110
Valtteri Bottas,+1000,+1000,+1000,+700,+5000,+1400,+800,+1200,+1200,+1100,+1400,+1600,+1600,+650,+350,+1600,+1100,+2000,+1600,+30000
Sergio Perez,+1400,+1100,+5000,+1600,+5000,+750,+2000,+1200,+1200,None,+1600,+1100,+2500,+3300,+2500,+1600,+1800,+1200,+2800,+4000
Pierre Gasly,+3300,+5000,+25000,+10000,+25000,+6000,+6000,+10000,+10000,+8000,+15000,None,+10000,+6600,+10000,+20000,+10000,+10000,+10000,+15000
Lando Norris,+5000,+2800,+10000,+5000,+10000,+1800,+10000,+3300,+3300,+4000,+3500,+2800,+4000,+3300,+2500,+3300,+4000,+4000,+6600,+2800
Charles Leclerc,+5000,+3300,+4000,+3300,+10000,+1600,+10000,+5000,+5000,+5000,+2200,+4000,+1600,+6600,+6600,+3300,+4000,+4000,+6600,+1200
Daniel Ricciardo,+5000,+5000,+10000,+5000,+50000,+4000,+20000,+10000,+10000,+8000,+8000,+20000,+10000,+10000,+2800,+6600,+10000,+6600,+10000,+6500
Yuki Tsunoda,+8000,+15000,+100000,+75000,+200000,+75000,None,+20000,+20000,+20000,+50000,+50000,None,+100000,+50000,None,+50000,+25000,+25000,+40000
Carlos Sainz Jr.,+10000,None,+10000,+6000,None,None,None,+8000,+8000,+6600,None,None,None,None,None,None,None,None,None,None


In [47]:
df.tail()

,Odds To Win 2021 Emilia Romagna GP (4/18/21),Odds To Win 2021 F1 Portuguese GP (3/28/21),Odds To Win 2021 F1 Spanish GP (5/9/21),Odds To Win 2021 F1 Monaco GP (3/28/21),Odds To Win 2021 F1 World Championship (3/28/21),Odds To Win 2021 F1 Azerbaijan GP (6/6/21),Odds To Win 2021 F1 French GP (6/20/21),Odds To Win 2021 F1 Austrian GP (6/26/21),Odds To Win 2021 F1 Austrian GP (7/4/21),Odds To Win 2021 F1 British GP (7/18/21),Odds To Win 2021 F1 Hungarian GP (8/1/21),Odds To Win 2021 F1 Belgium GP (8/29/21),Odds To Win 2021 F1 Dutch GP (9/5/21),Odds to Win 2021 F1 Italian GP (9/12/2021),Odds to Win 2021 F1 Russian GP (9/12/2021),Odds to Win 2021 F1 Turkish GP (10/10/2021),Odds to Win 2021 F1 U.S. GP (10/24/2021),Odds to Win 2021 F1 Mexico City GP (11/5/2021),Odds to Win 2021 F1 Abu Dhabi GP (12/12/2021),Odds to Win 2022 F1 World Championship (12/12/2022)
"Carlos Sainz, Jr.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None
Pierre Gasly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+25000,None,None,None,None,None,None,None,None
Robert Kubica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+200000,None,None,None,None,None,None
Alexander Albon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+50000
Guanyu Zhou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+100000


In [48]:
#ul = class_.find("ul")
#table = ul.findAll('li')
        


# as of 03/20/23 3pm, don't need to re-run any cell above; have all of the needed URLs in downloadedPagesChange
# using BeautifulSoup to read odds data into df (Pandas DataFrame defined above)

# len of downloadedPagesChange is 38
for page in downloadedPagesChange:
    # retrieve list of tables on the page
    canvas = BeautifulSoup(page.text, 'html.parser')
    try:
        allTables = canvas.findAll(class_="wp-block-table is-style-stripes") # UI changes again... need to find it
        race_title = canvas.find("h2").text
        rows = allTables[0].findAll('tr') # only want to scrape input from 1st table
        for j in range(len(rows)):
            if j > 0:
                dataPoints = rows[j].findAll('td')
                driver = dataPoints[0].text
                odds = dataPoints[1].text
                try:
                    df.loc[driver]
                except:
                    df.loc[driver] = None
                df.loc[driver, race_title] = odds
        #print('title '+race_title)
    except:
        pass
    
    try:
        class_ = canvas.find(class_='gutenberg__content wp-embed-responsive')
        
        ul = class_.find("ul")
        table = ul.findAll('li')
        
        race_title = canvas.find("h2").text
        #print(race_title)
        #header_info = class_.find("bf") # the entire header...
        #mytext = header_info.text.replace(header_info.find('strong').text, "") 
        #print(' date '+mytext)        
        # takes values from World Championship table for some reason
        #table = class_.findAll('ul', limit=1).findAll('li')
        for i in range(len(table)):
            driver_odds = table[i].text
            driver = driver_odds.rsplit(' ', 1)[0]
            odds = table[i].find("a").text
            try:
                df.loc[driver]
            except:
                df.loc[driver] = None
            
            #print(driver+' '+df.at[driver,race_title])
            df.loc[driver, race_title] = odds
    except:
        pass

#print(df.columns.tolist())
df

/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/1491332956.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[driver] = None
/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/1491332956.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[driver] = None
/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/1491332956.py:51: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[driver] = None
/var/folders/hj/2w1s9rpx58vgll96y2t7btpm0000gn/T/ipykernel_11434/1491332956.py:51: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Spec

,Odds To Win 2021 Emilia Romagna GP (4/18/21),Odds To Win 2021 F1 Portuguese GP (3/28/21),Odds To Win 2021 F1 Spanish GP (5/9/21),Odds To Win 2021 F1 Monaco GP (3/28/21),Odds To Win 2021 F1 World Championship (3/28/21),Odds To Win 2021 F1 Azerbaijan GP (6/6/21),Odds To Win 2021 F1 French GP (6/20/21),Odds To Win 2021 F1 Austrian GP (6/26/21),Odds To Win 2021 F1 Austrian GP (7/4/21),Odds To Win 2021 F1 British GP (7/18/21),...,2022 F1 BRITISH GP BETTING ODDS,2022 F1 AUSTRIAN GP BETTING ODDS,2022 F1 FRENCH GP BETTING ODDS,2022 F1 HUNGARIAN GP BETTING ODDS,2022 F1 BELGIAN GP BETTING ODDS,2022 F1 DUTCH GP BETTING ODDS,2022 F1 ITALIAN GP BETTING ODDS,2022 F1 SINGAPORE GP BETTING ODDS,2022 F1 JAPANESE GP BETTING ODDS,2022 F1 U.S. GP BETTING ODDS
Max Verstappen,+130,+140,+200,+140,+122,-110,+110,-148,-155,-160,...,-110,-120,+110,+110,-120,-175,-275,-250,-200,-250
Lewis Hamilton,+150,+160,-195,+100,-160,+250,+140,+200,+200,+185,...,+1100,+1000,+850,+1200,+500,+1200,+2000,+1200,+1000,+1200
Valtteri Bottas,+1000,+1000,+1000,+700,+5000,+1400,+800,+1200,+1200,+1100,...,+25000,+25000,+50000,+100000,+100000,+100000,+250000,+250000,+200000,+100000
Sergio Perez,+1400,+1100,+5000,+1600,+5000,+750,+2000,+1200,+1200,None,...,+900,+1000,+1400,+1400,+2000,+1000,+900,+1600,+1000,+800
Pierre Gasly,+3300,+5000,+25000,+10000,+25000,+6000,+6000,+10000,+10000,+8000,...,+40000,+40000,NaN,+75000,+100000,+100000,+200000,+75000,+100000,+100000
Lando Norris,+5000,+2800,+10000,+5000,+10000,+1800,+10000,+3300,+3300,+4000,...,+15000,+12500,+17500,+25000,+22500,+15000,+35000,+22500,+2000,+20000
Charles Leclerc,+5000,+3300,+4000,+3300,+10000,+1600,+10000,+5000,+5000,+5000,...,+225,+250,+137,+150,+350,+350,+500,+400,+350,+450
Daniel Ricciardo,+5000,+5000,+10000,+5000,+50000,+4000,+20000,+10000,+10000,+8000,...,+27500,+25000,+40000,+50000,+50000,+25000,+10000,+75000,+50000,+75000
Yuki Tsunoda,+8000,+15000,+100000,+75000,+200000,+75000,None,+20000,+20000,+20000,...,+100000,+50000,+100000,+150000,+200000,+200000,+250000,+200000,+200000,+150000
Carlos Sainz Jr.,+10000,None,+10000,+6000,None,None,None,+8000,+8000,+6600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# fixed Carlos Sainz common misspelling errors
for row in df.index:
    if "Carlos Sainz" in row and not row == "Carlos Sainz":
        for i, data in enumerate(df.loc[row]):
            if data is not None and not pd.isnull(data):
                df.loc["Carlos Sainz", df.columns[i]] = data
        df = df.drop(row)
df = df.drop("Odds to Win 2022 F1 World Championship (12/12/2022)", axis=1)

df

,Odds To Win 2021 Emilia Romagna GP (4/18/21),Odds To Win 2021 F1 Portuguese GP (3/28/21),Odds To Win 2021 F1 Spanish GP (5/9/21),Odds To Win 2021 F1 Monaco GP (3/28/21),Odds To Win 2021 F1 World Championship (3/28/21),Odds To Win 2021 F1 Azerbaijan GP (6/6/21),Odds To Win 2021 F1 French GP (6/20/21),Odds To Win 2021 F1 Austrian GP (6/26/21),Odds To Win 2021 F1 Austrian GP (7/4/21),Odds To Win 2021 F1 British GP (7/18/21),...,2022 F1 BRITISH GP BETTING ODDS,2022 F1 AUSTRIAN GP BETTING ODDS,2022 F1 FRENCH GP BETTING ODDS,2022 F1 HUNGARIAN GP BETTING ODDS,2022 F1 BELGIAN GP BETTING ODDS,2022 F1 DUTCH GP BETTING ODDS,2022 F1 ITALIAN GP BETTING ODDS,2022 F1 SINGAPORE GP BETTING ODDS,2022 F1 JAPANESE GP BETTING ODDS,2022 F1 U.S. GP BETTING ODDS
Max Verstappen,+130,+140,+200,+140,+122,-110,+110,-148,-155,-160,...,-110,-120,+110,+110,-120,-175,-275,-250,-200,-250
Lewis Hamilton,+150,+160,-195,+100,-160,+250,+140,+200,+200,+185,...,+1100,+1000,+850,+1200,+500,+1200,+2000,+1200,+1000,+1200
Valtteri Bottas,+1000,+1000,+1000,+700,+5000,+1400,+800,+1200,+1200,+1100,...,+25000,+25000,+50000,+100000,+100000,+100000,+250000,+250000,+200000,+100000
Sergio Perez,+1400,+1100,+5000,+1600,+5000,+750,+2000,+1200,+1200,None,...,+900,+1000,+1400,+1400,+2000,+1000,+900,+1600,+1000,+800
Pierre Gasly,+3300,+5000,+25000,+10000,+25000,+6000,+6000,+10000,+10000,+8000,...,+40000,+40000,NaN,+75000,+100000,+100000,+200000,+75000,+100000,+100000
Lando Norris,+5000,+2800,+10000,+5000,+10000,+1800,+10000,+3300,+3300,+4000,...,+15000,+12500,+17500,+25000,+22500,+15000,+35000,+22500,+2000,+20000
Charles Leclerc,+5000,+3300,+4000,+3300,+10000,+1600,+10000,+5000,+5000,+5000,...,+225,+250,+137,+150,+350,+350,+500,+400,+350,+450
Daniel Ricciardo,+5000,+5000,+10000,+5000,+50000,+4000,+20000,+10000,+10000,+8000,...,+27500,+25000,+40000,+50000,+50000,+25000,+10000,+75000,+50000,+75000
Yuki Tsunoda,+8000,+15000,+100000,+75000,+200000,+75000,None,+20000,+20000,+20000,...,+100000,+50000,+100000,+150000,+200000,+200000,+250000,+200000,+200000,+150000
George Russell,+20000,+100000,+250000,+200000,+300000,None,None,+50000,+50000,+50000,...,+1200,+1600,+1800,+2500,+1000,+1600,+900,+1600,+1600,+2500


In [55]:

df.to_csv(r'f1_betting_odds_2021-2022.csv')